In [2]:
import numpy as np
import copy
from tqdm import tqdm

In [3]:
# Load the graph problem from a .tsp.txt file
# data = np.loadtxt('data/eil51.tsp.txt', usecols=[1,2])
# data = np.loadtxt('test/test1.txt', usecols=[0,1])
# data

In [4]:
# Helper function to convert the coordinates into an adjacency matrix
def coordinates_to_adjacency_matrix(data,ord=2):
    a = np.zeros((len(data),len(data)))
    for i in range(len(a)):
        for j in range(len(a)):
            if not i == j:
                a[i][j] = np.linalg.norm(data[i] - data[j],ord=ord)
    return a

In [24]:
class Chromosome():
    
    # Random generated Chromosome
    #  m - number of traveling salesmans
    # def __init__(self, number_of_cities, number_of_traveling_salesman, adj = coordinates_to_adjacency_matrix(data)):
    def __init__(self, number_of_cities, number_of_traveling_salesman, adj):
        self.n = number_of_cities
        self.m = number_of_traveling_salesman
        self.adj = adj
        c = np.array(range(1,number_of_cities))
        np.random.shuffle(c)
        self.solution = np.array_split(c, self.m)
        for i in range(len(self.solution)):
            self.solution[i] = np.insert(self.solution[i],0,0)
            self.solution[i] = np.append(self.solution[i],0)
        self.fitness()
            
    # Evaluate the Chromosome - Fitness function
    #  based on 2 features: 
    #   - overall cost (cumulated from all salesman)
    #   - worst (longest) salesman cost
    #  adj - adjacency matrix
    def fitness(self):
        self.cost = 0
        self.minmax=0
        # longest_salesman_fitness = []
        # longest_salesman_length = 0
        for i in range(self.m):
            salesman = self.solution[i]
            salesman_fitness = 0
            for j in range(len(salesman) - 1):
                salesman_fitness = salesman_fitness + self.adj[salesman[j]][salesman[j+1]]
            self.cost = self.cost + salesman_fitness
            # if len(salesman) > longest_salesman_length or (len(salesman) == longest_salesman_length and salesman_fitness > self.minmax):
            #     longest_salesman_length = len(salesman)
            #     self.minmax = salesman_fitness
            # if len(salesman) > longest_salesman_length:
            #     longest_salesman_length = len(salesman)
            if salesman_fitness > self.minmax:
                self.minmax = salesman_fitness
        self.score =  self.minmax
        # printChromesome(self)

    # Mutation operator - mutates a single Traveling Salesman
    #  by swaping 2 cities
    def mutate_local(self):
        index = np.random.randint(0,self.m)
        mutant = self.solution[index]
        i,j = np.random.randint(1,len(mutant)-1), np.random.randint(1,len(mutant)-1)
        mutant[i], mutant[j] = mutant[j], mutant[i]
        old_cost = self.cost
        self.fitness()
    
    # Mutation operator - mutates 2 Traveling Salesmans
    #  by removing a city from a salesman and asigning it to the second one
    def mutate_global(self):
        for i in range(self.m):
            if len(self.solution[i]) < 3:
                print(i, self.solution[i])
        
        
        index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while index1 == index2:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while len(self.solution[index1]) < 4:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        mutant1, mutant2 = self.solution[index1], self.solution[index2]
        i,j = np.random.randint(1,len(mutant1)-1), np.random.randint(1,len(mutant2)-1)
        self.solution[index2] = np.insert(mutant2, j, mutant1[i])
        self.solution[index1] = np.delete(mutant1, i)
        old_cost = self.cost
        self.fitness()

    def printChromesome(self):
        total_cost = 0
        minmax = 0
        for i in range(self.m):
            salesman = self.solution[i]
            cost=0
            print(i+1, ":  ", self.solution[i][0]+1, end="", sep="")
            for j in range(1,len(self.solution[i])):
                # print("-", self.solution[i][j]+1, end="", sep="")
                dist=self.adj[salesman[j-1]][salesman[j]]
                print("[%.0f]%d"%(dist,self.solution[i][j]+1), end="", sep="")
                cost+=dist
            total_cost+=cost
            if cost>minmax:
                minmax = cost
            print(" --- %.0f#"%(cost), len(self.solution[i]))
        # print("Cost:   \t%.1f\t%.1f"%(self.cost,total_cost))
        # print("Minmax: \t%.1f\t%.1f"%(self.minmax,minmax))
        print("Cost:   \t%.1f"%(total_cost))
        print("Minmax: \t%.1f"%(minmax))

    

In [25]:
def optimize(n_of_ts,coordinates,order,cycle=100000):
    adjacency = coordinates_to_adjacency_matrix(coordinates,ord=order)
    n_cities = len(coordinates)
    # print("n_cities:",n_cities)
    chromosome = Chromosome(number_of_cities = n_cities, number_of_traveling_salesman = n_of_ts, adj=adjacency)
    for it in tqdm(range(cycle)):
        # Mutate globally
        chromosome_copy = copy.deepcopy(chromosome)
        chromosome_copy.mutate_global()
        if chromosome_copy.score < chromosome.score:
            chromosome = chromosome_copy
        # Mutate locally
        chromosome_copy = copy.deepcopy(chromosome)
        chromosome_copy.mutate_local()
        if chromosome_copy.score < chromosome.score:
            chromosome = chromosome_copy
    return(chromosome)

In [22]:
data = np.loadtxt('data/eil51.tsp.txt', usecols=[1,2])
# data = np.loadtxt('test/test1.txt', usecols=[0,1])
data

ch=optimize(n_of_ts=2,coordinates=data,order=1,cycle=100000)

100%|██████████| 100000/100000 [00:22<00:00, 4402.61it/s]


In [23]:
ch.printChromesome()

1:  1[17]31[43]12[20]11[15]2[71]41[23]18[18]6[33]43[70]42[18]45[9]15[10]37[40]48[30]28[26]32[27]47[23]38[25]22[16]20[22]16[8]9[9]49[31]44[46]25[15]24[33]8[20]23[51]5[25]1

NameError: name 'chromosome' is not defined